In [ ]:
#=================================================================================================
# restAPI 테스트
# =>SBERT를 활용한 임베딩 Flask 서버 구축후, 문장을 입력 후 임베딩 값을 얻어온다.
# =>curl로 테스트 시에는 cmd창에서 아래 처럼 한글 text(안녕하세요)는 인코딩된 값으로 입력해야 함.
# curl -d "{""text"":""%EC%95%88%EB%85%95%ED%95%98%EC%84%B8%EC%9A%94""}" 
# -H "Content-Type: application/json" 
# -X POST http://127.0.0.1:9999/embed
#=================================================================================================
import requests
import json

In [ ]:
# 임베딩 예시
# => curl -d "{""text"":""%EC%95%88%EB%85%95%ED%95%98%EC%84%B8%EC%9A%94""}" -H "Content-Type: application/json" -X POST http://127.0.0.1:9999/embed
FLASK_URL = "http://127.0.0.1:9999/embed"

def run_embedding_query_loop():
    while True:
        try:
            handle_query_embedding()
        except KeyboardInterrupt:
            return
        
def handle_query_embedding():
    #url
    url = FLASK_URL

    #headers
    headers = {
        "Content-Type": "application/json"
    }

    #data
    #query = "my name is bongsoo. who are you?"
    #query = "안녕하세요.지금은 curl 테스트 입니다."
    query = input("임베딩할 문장 입력: ")

    # json 구조
    temp = {
        "text": query,
        #"text2": text2,
    }

    # dict 을 json으로 변환
    data = json.dumps(temp)
    
    response = requests.post(url, headers=headers, data=data)
    print("response: ", response)
    print(type(response.text))
    print("response.text: ", response.text)
    

In [ ]:
run_embedding_query_loop()

In [ ]:
# 문장 요약후 임베딩 예시
# => curl -d "{""text"":""%EC%95%88%EB%85%95%ED%95%98%EC%84%B8%EC%9A%94""}" -H "Content-Type: application/json" -X POST http://127.0.0.1:9999/embed?min_length=10&num_sentence=2

MIN_LENGTH = 10
NUM_SENTENCE = 2
FLASK_URL = f"http://127.0.0.1:9999/summarize?min_length={MIN_LENGTH}&num_sentence={NUM_SENTENCE}"

def run_summarize_query_loop():
    while True:
        try:
            handle_query_summarize()
            #return
        except KeyboardInterrupt:
            return
        
def handle_query_summarize():
    #url
    url = FLASK_URL

    #headers
    headers = {
        "Content-Type": "application/json"
    }

    #data
    #query = "my name is bongsoo. who are you?"
    
    #query = '''
    #대한민국의 남서쪽에 있는 섬. 행정구역상 광역자치단체인 제주특별자치도의 관할. 
    #한국의 섬 중에서 가장 크고 인구가 많은 섬으로 면적은 1833.2㎢이다. 제주도 다음 2번째 큰 섬인 거제도의 5배 정도 된다. 인구는 약 68만 명.
    #'''
    query = input("요약할 문장 입력: ")

    # json 구조
    temp = {
        "text": query,
        #"text2": text2,
    }

    # dict 을 json으로 변환
    data = json.dumps(temp)
    
    response = requests.post(url, headers=headers, data=data)
    print("response: ", response)
    print("response.text: ", response.text)

In [ ]:
run_summarize_query_loop()

In [ ]:
# ES vector 검색 예시
# => curl -d "{""text"":""%EC%95%88%EB%85%95%ED%95%98%EC%84%B8%EC%9A%94""}" -H "Content-Type: application/json" -X POST http://127.0.0.1:9999/search?eshost='http://192.168.0.27:9200/'&index='korquad-klue-2'&size=5

ESURL = 'http://192.168.0.27:9200/'
INDEX_NAME = 'korquad-klue-2'
SEARCH_SIZE = 4

FLASK_URL = f"http://127.0.0.1:9999/search?eshost={ESURL}&index={INDEX_NAME}&size={SEARCH_SIZE}"

def run_search_query_loop():
    while True:
        try:
            handle_query_search()
            #return
        except KeyboardInterrupt:
            return
        
def handle_query_search():
    #url
    url = FLASK_URL

    #headers
    headers = {
        "Content-Type": "application/json"
    }

    #data
    #query = "세상에게 귀여운 동물들들"
    query = input("검색할 문장 입력: ")

    # json 구조
    temp = {
        "text": query,
        #"text2": text2,
    }

    # dict 을 json으로 변환
    data = json.dumps(temp)
    
    response = requests.post(url, headers=headers, data=data)
    print("response: ", response)
    #print("response.text: ", response.text)
    
    # 들어온 응답 결과 str -> dict 로 변환하기 위해 json.loads 이용
    search_result = json.loads(response.text)
    print(type(search_result))
    #print(search_result)
    
    
    # 쿼리 응답 결과값에서 _id, _score, _source 등을 뽑아냄
    for hit in search_result["hits"]["hits"]: 
        
        print("index:{}, type:{}".format(hit["_index"], hit["_type"]))
        print("id: {}, score: {}".format(hit["_id"], hit["_score"])) 
        
        print(f'[제목] {hit["_source"]["title"]}')
        
        print('[요약문]')
        print(hit["_source"]["summarize"]) 
        
        print()   

In [ ]:
run_search_query_loop()